# ***** NOTEBOOK 1 : Analyse Exploratoire des données principales*******

# 1 Importation des librairies

In [ ]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
#pd.options.plotting.backend = "plotly"
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import plotly as py
py.offline.init_notebook_mode(connected=True)
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.offline as offline
import cufflinks as cf
cf.go_offline()
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row',250)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

# 2 Importation jeux de données

In [ ]:
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

application_train.head()

# 3. Traitement des variables non numériques
Les machines ne peuvent comprendre que les chiffres. Convertissons donc toutes les colonnes non numériques en nombres. Les variables catégorielles seront converties en colonnes fictives, les variables ordinales sont converties en nombres par mappage et les variables qui ne sont pas numériques et ne peuvent pas être converties en nombres seront supprimées du modèle.

In [ ]:
#List of non-numerical variables
application_train.select_dtypes(include=['O']).columns

In [ ]:
#Nous ne pouvons pas avoir de colonnes non numériques pour la modélisation. Nous ne pouvons avoir que des colonnes numériques. Les colonnes non numériques peuvent également être des variables ordinales ou catégorielles.
col_for_dummies=application_train.select_dtypes(include=['O']).columns.drop(['FLAG_OWN_CAR','FLAG_OWN_REALTY','EMERGENCYSTATE_MODE'])
application_train_dummies = pd.get_dummies(application_train, columns = col_for_dummies, drop_first = True)
application_test_dummies = pd.get_dummies(application_test, columns = col_for_dummies, drop_first = True)

In [ ]:
application_train_dummies.select_dtypes(include=['O']).columns

In [ ]:
application_train_dummies['EMERGENCYSTATE_MODE'].value_counts()

In [ ]:
#Nous ne pouvons pas convertir flag_own_car et flag_own_realty en colonne avec oui ou non, etc. Faisons plutôt correspondre oui à 1 et non à 0
application_train_dummies['FLAG_OWN_CAR'] = application_train_dummies['FLAG_OWN_CAR'].map( {'Y':1, 'N':0})
application_train_dummies['FLAG_OWN_REALTY'] = application_train_dummies['FLAG_OWN_REALTY'].map( {'Y':1, 'N':0})
application_train_dummies['EMERGENCYSTATE_MODE'] = application_train_dummies['EMERGENCYSTATE_MODE'].map( {'Yes':1, 'No':0})

application_test_dummies['FLAG_OWN_CAR'] = application_train_dummies['FLAG_OWN_CAR'].map( {'Y':1, 'N':0})
application_test_dummies['FLAG_OWN_REALTY'] = application_train_dummies['FLAG_OWN_REALTY'].map( {'Y':1, 'N':0})
application_test_dummies['EMERGENCYSTATE_MODE'] = application_train_dummies['EMERGENCYSTATE_MODE'].map( {'Yes':1, 'No':0})
print(application_train_dummies.shape)
print(application_test_dummies.shape)

In [ ]:
#Nous avons 4 colonnes de moins dans application_test_dummies. Voyons quelles sont ces 4 colonnes
#Parfois, les données de test n'ont pas certaines colonnes.
application_train_dummies.columns.difference(application_test_dummies.columns)

# 4. Alignement des jeux de donnés d'entrainement et test

In [ ]:
#Il doit y avoir les mêmes caractéristiques (colonnes) dans les données d'entrainement et de test. L'encodage à chaud a créé plus de colonnes dans les données d'apprentissage car il y avait des variables catégorielles avec des catégories non représentées dans les données de test. Pour supprimer les colonnes des données d'entraînement qui ne figurent pas dans les données de test, nous devons aligner les dataframes. Nous extrayons d'abord la colonne cible des données d'apprentissage (car cela ne figure pas dans les données de test, mais nous devons conserver ces informations). Lorsque nous faisons l'alignement, nous devons nous assurer de définir axis = 1 pour aligner les dataframes en fonction des colonnes et non des lignes !
train_labels = application_train_dummies['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
application_train_dummies, application_test_dummies = application_train_dummies.align(application_test_dummies, join = 'inner', axis = 1)

# Add the target back in
application_train_dummies['TARGET'] = train_labels

print('Training Features shape: ', application_train_dummies.shape)
print('Testing Features shape: ', application_test_dummies.shape)

###### The training and testing datasets now have the same features which is required for machine learning. The number of features has grown significantly due to one-hot encoding. At some point we probably will want to try dimensionality reduction (removing features that are not relevant) to reduce the size of the datasets.

# 5 Traitement des valeurs manquantes (à l'aide d'un imputer itératif) avant  détection des valeurs aberrantes
Nous devons gérer nos valeurs manquantes avant de pouvoir effectuer tout type de détection de valeurs aberrantes. Il existe de nombreuses façons de gérer les valeurs manquantes. Nous pouvons utiliser fillna() et remplacer les valeurs manquantes par la moyenne, la médiane ou la valeur la plus fréquente des données. L'approche que nous utiliserons ci-dessous sera Iterative Imputer. L'imputateur itératif considérera la variable manquante comme la variable dépendante et toutes les autres caractéristiques seront des variables indépendantes. Il y aura donc une régression et les variables indépendantes seront utilisées pour déterminer la variable dépendante (qui est la caractéristique manquante).

In [ ]:
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
import random

In [ ]:
y=application_train_dummies[['SK_ID_CURR','TARGET']]
X=application_train_dummies.drop(columns=['TARGET'], axis=1)
X.head()

In [ ]:
X_imputation = X.loc[:, (X.nunique() > 1000)]

In [ ]:
X_imputation.columns

In [ ]:
imputer = IterativeImputer(BayesianRidge())
imputed_total = pd.DataFrame(imputer.fit_transform(X_imputation))
imputed_total.columns = X_imputation.columns

# 6 Détection des valeurs aberrantes
En statistique, une valeur aberrante est un point d'observation éloigné des autres observations. Il existe de nombreuses façons de détecter les valeurs aberrantes.

Méthodes visuelles pour repérer et supprimer les valeurs aberrantes

Boîte à moustaches
Nuages ​​de points
Détection et suppression des valeurs aberrantes à l'aide d'une fonction mathématique

Z-score : un seuil de -3 à 3 est pris, et tout point avec un score z non compris dans cette plage est supprimé en tant que valeur aberrante.
Score IQR : Cela fonctionne comme un diagramme en boîte et un score z en ce sens qu'une valeur seuil IQR est définie. L'IQR est le premier quartile soustrait du troisième quartile. Tout point en dessous du seuil IQR est supprimé.

##### Méthodes de classification pour la détection des valeurs aberrantes

Clustering DBScan (création de clusters autour de points de données). Un nombre minimum de points est requis pour être dans un cluster. Il y aura des points qui n'appartiennent à aucun cluster ou bien des points qui sont uniques dans un cluster entier. Nous pouvons donc supprimer ces points de bruit.
Forêt d'isolement : la forêt d'isolement produira les prédictions pour chaque point de données dans un tableau. Si le résultat est -1, cela signifie que ce point de données spécifique est une valeur aberrante. Si le résultat est 1, cela signifie que le point de données n'est pas une valeur aberrante
Ici, nous utiliserons la méthode Isolation Forest car elle peut bien gérer les valeurs manquantes et ne nécessite pas de mise à l'échelle des entrées

In [ ]:
from sklearn.ensemble import IsolationForest
rs=np.random.RandomState(0)
clf = IsolationForest(max_samples=100,random_state=rs, contamination=.1) 
clf.fit(imputed_total)
if_scores = clf.decision_function(imputed_total)

pred = clf.predict(imputed_total)
imputed_total['anomaly']=pred
outliers=imputed_total.loc[imputed_total['anomaly']==-1]
outlier_index=list(outliers.index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(imputed_total['anomaly'].value_counts())

In [ ]:
outlier_ID=list(outliers['SK_ID_CURR'])
X_new = X[~X.SK_ID_CURR.isin(outlier_ID)]
y_new = y[~y.SK_ID_CURR.isin(outlier_ID)]

In [ ]:
print(X_new.shape)
print(X.shape)

### 6.1 Détection des anomalies

Bien que nous ayons supprimé les valeurs aberrantes à l'aide de la forêt d'isolement, nous verrons toujours les données une fois pour vérifier toute anomalie. La forêt d'isolement ou toute méthode de détection de valeurs aberrantes suppose que la valeur aberrante est un point minoritaire et ne ressemble pas aux autres points majoritaires. Cependant, certains points d'aberration sont parfois trop nombreux. Voyons s'il existe une telle anamolie que nous trouvons


In [ ]:
X_new.describe()

# 7 Détection univariée des valeurs aberrantes

##### Negative numbers: DAYS_BIRTH, DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH, DAYS_LAST_PHONE_CHANGE
Univariate outliers detection

Negative numbers: DAYS_BIRTH, DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH, DAYS_LAST_PHONE_CHANGE

Numbers are negative since they are taken relative to the date of application. So we need to change them to positive.

#### Maximum value discrepancy Days_EMPLOYED:365243 days(over 1000years) OWN_CAR_AGE:91 yEARS

In [ ]:
#Checking the anamalous variables values in years
print('DAYS_BIRTH stats in years:','\n',(X_new['DAYS_BIRTH'] / -365).describe(),'\n')
print('Check the stats in years to see if there is any anomalous behavior')
print('DAYS_EMPLOYED stats in years:','\n',(X_new['DAYS_EMPLOYED'] / -365).describe(),'\n')
print('DAYS_REGISTRATION stats in years:','\n',(X_new['DAYS_REGISTRATION'] / -365).describe(),'\n')
print('DAYS_ID_PUBLISH stats in years:','\n',(X_new['DAYS_ID_PUBLISH'] / -365).describe(),'\n')
print('DAYS_LAST_PHONE_CHANGE stats in years:','\n',(X_new['DAYS_LAST_PHONE_CHANGE'] / -365).describe(),'\n')

Comme nous pouvons le voir, il y a une anomalie dans Days_employed car il est très peu probable qu'une personne soit employée pendant 1000 ans.

In [ ]:
X_new['DAYS_EMPLOYED'].max()

In [ ]:
# Replace the error values in Days_employed with nan
X_new['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
application_test_dummies['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [ ]:
X_new.describe()

Les données sont maintenant belles et propres.

# 8.Données manquantes dans la table de donné application_train

In [ ]:
# checking missing data
total = X_new.isnull().sum().sort_values(ascending = False)
percent = (X_new.isnull().sum()/X_new.isnull().count()*100).sort_values(ascending = False)
missing_application_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_application_train_data.head(20)

# 9.Duplicate data in application_train

In [ ]:
columns_without_id = [col for col in X_new.columns if col!='SK_ID_CURR']
#Checking for duplicates in the data.
X_new[X_new.duplicated(subset = columns_without_id, keep=False)]
print('The no of duplicates in the data:',X_new[X_new.duplicated(subset = columns_without_id, keep=False)]
      .shape[0])

# 10 Vérification du déséquilibre des données cible

In [ ]:
y_new['TARGET'].value_counts()

On voit que la classe est clairement déséquilibrée avec des cas de défaut très faibles par rapport à l'ensemble des cas. Nous devons donc équilibrer les données lorsque nous utilisons des modèles d'apprentissage automatique.

In [ ]:
Des=pd.DataFrame(y_new['TARGET'].value_counts())

In [ ]:
Des

In [ ]:
import plotly.express as px

In [ ]:
labels = 'Loan Secured', 'Loan Unsecured'
sizes = Des.TARGET
colors = ['lightskyblue','lightcoral']
fig = plt.figure(figsize =(3, 3))
plt.pie(sizes, labels=labels, colors=colors, 
        autopct='%1.1f%%', shadow=True, startangle=90)

plt.axis('equal')

plt.savefig('PieChart01.png')
plt.show()

In [ ]:
X_new.to_csv('X_new.csv',index=False)

In [ ]:
y_new.to_csv('y_new.csv',index=False)

# 11.Analyse exploratoire par visualisation de la distribution des variables de la table de donnée principale: application_train 

In [ ]:
X_new.head()

## 11.1 Distribution des revenus

In [ ]:
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

plt.figure(figsize=(12,5))
plt.title("Distribution of AMT_INCOME_TOTAL")
ax = sns.distplot(X_new["AMT_INCOME_TOTAL"])

La distribution est asymétrique à droite et il y a des valeurs extrêmes, nous pouvons appliquer la distribution logarithmique.

In [ ]:
X_new["AMT_INCOME_TOTAL"].describe()

In [ ]:
application_train=pd.merge(X_new,y_new,on='SK_ID_CURR')

In [ ]:
(application_train[application_train['AMT_INCOME_TOTAL'] > 1000000]['TARGET'].value_counts())/len(application_train[application_train['AMT_INCOME_TOTAL'] > 1000000])*100

Les personnes à revenu élevé ont tendance à ne pas etre en  défaut

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
#boxcox=0 means we are taking log transformation of data to show it as normal form

from scipy.stats import boxcox
from matplotlib import pyplot


np.log(application_train['AMT_INCOME_TOTAL']).iplot(kind='histogram', bins=100,
                               xTitle = 'log(INCOME_TOTAL)',yTitle ='Count corresponding to Incomes',
                               title='Distribution of log(AMT_INCOME_TOTAL)')

Nous voyons que la variable de revenu obtient une distribution normale lorsqu'elle est transformée en log.

## 11.2. Répartition du crédit

In [ ]:
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

plt.figure(figsize=(12,5))
plt.title("Distribution of AMT_CREDIT")
ax = sns.distplot(application_train["AMT_CREDIT"])

In [ ]:
application_train["AMT_CREDIT"].describe()

In [ ]:
(application_train[application_train['AMT_CREDIT']>2000000]['TARGET'].value_counts())/len(application_train[application_train['AMT_CREDIT']>2000000])*100

Les gens qui prennent moins de défaut de crédit

## 11.3. Répartition des types de prêts

In [ ]:
original_train_data = pd.read_csv('application_train.csv')

contract_val = original_train_data['NAME_CONTRACT_TYPE'].value_counts()
contract_df = pd.DataFrame({'labels': contract_val.index,
                   'values': contract_val.values
                  })
contract_df.iplot(kind='pie',labels='labels',values='values', title='Types of Loan')

Plus de gens sont intéressés à contracter des prêts en espèces que des prêts renouvelables.

## 11.4 Distribution de type de revenues

In [ ]:
original_train_data["NAME_INCOME_TYPE"].iplot(kind="histogram", bins=20, theme="white", title="Passenger's Income Types",
                                            xTitle='Name of Income Types', yTitle='Count')

In [ ]:
education_val = original_train_data['NAME_INCOME_TYPE'].value_counts()

education_val_y0 = []
education_val_y1 = []
for val in education_val.index:
    education_val_y1.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_INCOME_TYPE']==val] == 1))
    education_val_y0.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_INCOME_TYPE']==val] == 0))

data = [go.Bar(x = education_val.index, y = ((education_val_y1 / education_val.sum()) * 100), name='Default' ),
        go.Bar(x = education_val.index, y = ((education_val_y0 / education_val.sum()) * 100), name='No default' )]

layout = go.Layout(
    title = "Income of people affecting default on loans",
    xaxis=dict(
        title='Income of people',
       ),
    yaxis=dict(
        title='Count of people accompanying in %',
        )
)

fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'plotly_dark'
fig.show()

## 11.5 Distribution du Type d'accompagnateur

Qui a accompagné la personne lors de la souscription du prêt ?

In [ ]:
original_train_data["NAME_TYPE_SUITE"].iplot(kind="histogram", bins=20, theme="white", title="Accompanying Person",
                                            xTitle='People accompanying', yTitle='Count')

Most people are unaccompanied.

## 11.6. Distribution du nom du type d'éducation

In [ ]:
education_val = original_train_data['NAME_EDUCATION_TYPE'].value_counts()

education_val_y0 = []
education_val_y1 = []
for val in education_val.index:
    education_val_y1.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_EDUCATION_TYPE']==val] == 1))
    education_val_y0.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_EDUCATION_TYPE']==val] == 0))

data = [go.Bar(x = education_val.index, y = ((education_val_y1 / education_val.sum()) * 100), name='Default' ),
        go.Bar(x = education_val.index, y = ((education_val_y0 / education_val.sum()) * 100), name='No default' )]

layout = go.Layout(
    title = "Education sources of Applicants in terms of loan is repayed or not  in %",
    xaxis=dict(
        title='Education of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)

fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'plotly_dark'
fig.show()

Les titulaires d'un diplôme sont en mesure de rembourser la plupart du temps.

## 11.7 Effet de l'état matrimonial sur la capacité de rembourser les prêts

In [ ]:
education_val = original_train_data['NAME_FAMILY_STATUS'].value_counts()

education_val_y0 = []
education_val_y1 = []
for val in education_val.index:
    education_val_y1.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_FAMILY_STATUS']==val] == 1))
    education_val_y0.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_FAMILY_STATUS']==val] == 0))

data = [go.Bar(x = education_val.index, y = ((education_val_y1 / education_val.sum()) * 100), name='Default' ),
        go.Bar(x = education_val.index, y = ((education_val_y0 / education_val.sum()) * 100), name='No default' )]

layout = go.Layout(
    title = "Family status of Applicant in terms of loan is repayed or not in %",
    xaxis=dict(
        title='Family status of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)

fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'plotly_dark'
fig.show()

## 11.8. Distribution du Type de Résidence

In [ ]:
education_val = original_train_data['NAME_HOUSING_TYPE'].value_counts()

education_val_y0 = []
education_val_y1 = []
for val in education_val.index:
    education_val_y1.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_HOUSING_TYPE']==val] == 1))
    education_val_y0.append(np.sum(original_train_data['TARGET'][original_train_data['NAME_HOUSING_TYPE']==val] == 0))

data = [go.Bar(x = education_val.index, y = ((education_val_y1 / education_val.sum()) * 100), name='Default' ),
        go.Bar(x = education_val.index, y = ((education_val_y0 / education_val.sum()) * 100), name='No default' )]

layout = go.Layout(
    title = "Housing Type of Applicant in terms of loan is repayed or not in %",
    xaxis=dict(
        title='Housing Type of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)

fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'plotly_dark'
fig.show()

Les gens dans un appartement de bureau, un appartement coopératif ne font presque jamais défaut.

## 11.9 Distribution de l'age

In [ ]:
(original_train_data["DAYS_BIRTH"]/-365).iplot(kind="histogram", bins=20, theme="white", title="Customer's Ages",
                                            xTitle='Age of customer', yTitle='Count')

## 11.10 Analyse visuelle du défaut en fonction de  la variable  OCCUPATION_TYPE(Statut Professionnel) 

In [ ]:
parameter_val = original_train_data['OCCUPATION_TYPE'].value_counts()

parameter_val_y0 = []
parameter_val_y1 = []
for val in parameter_val.index:
    parameter_val_y1.append(np.sum(original_train_data['TARGET'][original_train_data['OCCUPATION_TYPE']==val] == 1))
    parameter_val_y0.append(np.sum(original_train_data['TARGET'][original_train_data['OCCUPATION_TYPE']==val] == 0))

data = [go.Bar(x = parameter_val.index, y = ((parameter_val_y1 / parameter_val.sum()) * 100), name='Default' ),
        go.Bar(x = parameter_val.index, y = ((parameter_val_y0 / parameter_val.sum()) * 100), name='No default' )]

layout = go.Layout(
    title = "Occupation type of people affecting default on loans",
    xaxis=dict(
        title='Occupation type of people',
       ),
    yaxis=dict(
        title='Count of people Occupation that type of housing in %',
        )
)

fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'plotly_dark'
fig.show()

Les personnes hautement qualifiées sont plus susceptibles de rembourser et les personnes peu qualifiées moins susceptibles de rembourser leurs prêts

In [ ]:
application_train_dummies.head()

 # 12 Processus de combinaison des tables de données et extraction des données supplémentaires

## 12.1 Features Engineering

### 12.1.1 Feature Engineering de la table de données Application

In [ ]:
#Flag to represent when credit > income
#Train
application_train_dummies['Credit_flag'] = application_train_dummies['AMT_INCOME_TOTAL'] > application_train_dummies['AMT_CREDIT']
application_train_dummies['Percent_Days_employed'] = application_train_dummies['DAYS_EMPLOYED']/application_train_dummies['DAYS_BIRTH']*100
application_train_dummies['Annuity_as_percent_income'] = application_train_dummies['AMT_ANNUITY']/ application_train_dummies['AMT_INCOME_TOTAL']*100
application_train_dummies['Credit_as_percent_income'] = application_train_dummies['AMT_CREDIT']/application_train_dummies['AMT_INCOME_TOTAL']*100
#Test
application_test_dummies['Credit_flag'] = application_test_dummies['AMT_INCOME_TOTAL'] > application_test_dummies['AMT_CREDIT']
application_test_dummies['Percent_Days_employed'] = application_test_dummies['DAYS_EMPLOYED']/application_test_dummies['DAYS_BIRTH']*100
application_test_dummies['Annuity_as_percent_income'] = application_test_dummies['AMT_ANNUITY']/ application_test_dummies['AMT_INCOME_TOTAL']*100
application_test_dummies['Credit_as_percent_income'] = application_test_dummies['AMT_CREDIT']/application_test_dummies['AMT_INCOME_TOTAL']*100

### 12.1.2 Feature engineering de la table de données Bureau

In [ ]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [ ]:
# Combining numerical features
bureau = pd.read_csv('bureau.csv')
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
application_bureau = application_train_dummies.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = application_test_dummies.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))

In [ ]:
# Combining categorical features
bureau_categorical = pd.get_dummies(bureau.select_dtypes('object'))
bureau_categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
grp = bureau_categorical.groupby(by = ['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))

### 12.1.3 Feature Engineering de la table de données Bureau Data

In [ ]:
# Number of types of past loans per customer 
grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_TYPES'] = application_bureau['BUREAU_LOAN_TYPES'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_TYPES'] = application_bureau_test['BUREAU_LOAN_TYPES'].fillna(0)

In [ ]:
# Debt over credit ratio 
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM': 'TOTAL_CREDIT_SUM'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CREDIT_SUM_DEBT'})

grp1['DEBT_CREDIT_RATIO'] = grp2['TOTAL_CREDIT_SUM_DEBT']/grp1['TOTAL_CREDIT_SUM']

del grp1['TOTAL_CREDIT_SUM']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['DEBT_CREDIT_RATIO'] = application_bureau['DEBT_CREDIT_RATIO'].fillna(0)
application_bureau['DEBT_CREDIT_RATIO'] = application_bureau['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0)
application_bureau['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau['DEBT_CREDIT_RATIO'], downcast='float')

application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['DEBT_CREDIT_RATIO'] = application_bureau_test['DEBT_CREDIT_RATIO'].fillna(0)
application_bureau_test['DEBT_CREDIT_RATIO'] = application_bureau_test['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0)
application_bureau_test['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau_test['DEBT_CREDIT_RATIO'], downcast='float')

In [ ]:
# Overdue over debt ratio
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CUSTOMER_DEBT'})

grp1['OVERDUE_DEBT_RATIO'] = grp1['TOTAL_CUSTOMER_OVERDUE']/grp2['TOTAL_CUSTOMER_DEBT']

del grp1['TOTAL_CUSTOMER_OVERDUE']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['OVERDUE_DEBT_RATIO'] = application_bureau['OVERDUE_DEBT_RATIO'].fillna(0)
application_bureau['OVERDUE_DEBT_RATIO'] = application_bureau['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0)
application_bureau['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau['OVERDUE_DEBT_RATIO'], downcast='float')

application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['OVERDUE_DEBT_RATIO'] = application_bureau_test['OVERDUE_DEBT_RATIO'].fillna(0)
application_bureau_test['OVERDUE_DEBT_RATIO'] = application_bureau_test['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0)
application_bureau_test['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau_test['OVERDUE_DEBT_RATIO'], downcast='float')

In [ ]:
import gc

gc.collect()

### 12.1.4 Feature engineering de la table de données Previous Application

In [ ]:
def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    return first + second == 2
previous_application = pd.read_csv('previous_application.csv')
isOneToOne(previous_application,'SK_ID_CURR','SK_ID_PREV')

In [ ]:
# Number of previous applications per customer
grp = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})

# Take only the IDs which are present in application_bureau
application_bureau_prev = application_bureau.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test = application_bureau_test.merge(grp, on =['SK_ID_CURR'], how = 'left')

#Fill NA for previous application counts (lets say there was an application ID present in application_bureau but not present
# in grp, then that means that person never took loan previously, so count of previous loan for that person = 0)
application_bureau_prev['PREV_APP_COUNT'] = application_bureau_prev['PREV_APP_COUNT'].fillna(0)
application_bureau_prev_test['PREV_APP_COUNT'] = application_bureau_prev_test['PREV_APP_COUNT'].fillna(0)

In [ ]:
# Combining numerical features

#Take the mean of all the parameters (grouping by SK_ID_CURR)
grp = previous_application.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()

#Add prefix prev in front of all columns so that we know that these columns are from previous_application
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]

#Change the columns
grp.columns = prev_columns

application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [ ]:
gc.collect()

### 12.1.5 Feature engineering  de la table de données installments_payments

In [ ]:
insta_payments = pd.read_csv('installments_payments.csv')

In [ ]:
# Combining numerical features and there are no categorical features in this dataset
grp = insta_payments.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTA_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [ ]:
gc.collect()

### 12.1.6 Feature engineering  de la table de données Credit card balance

In [ ]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [ ]:
credit_card=credit_card_balance
# Combining numerical features
grp = credit_card.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [ ]:
#Combining categorical features
credit_categorical = pd.get_dummies(credit_card.select_dtypes('object'))
credit_categorical['SK_ID_CURR'] = credit_card['SK_ID_CURR']

grp = credit_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]

application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [ ]:
application_bureau_prev.head()

In [ ]:
application_bureau_prev_test.head()

# 13. Application Train_agg/Test_agg--->Data Final  Processing

In [ ]:
appli_train_agg=application_bureau_prev.copy()
appli_test_agg=application_bureau_prev_test.copy()

In [ ]:
train_Sign=appli_train_agg['SK_ID_CURR']
test_Sign=appli_test_agg['SK_ID_CURR']

## 13.1 Séparation de la variable Target du training dataset aggrégé

In [ ]:
y_train_agg = appli_train_agg['TARGET']
X = appli_train_agg.drop('TARGET', axis = 1)

In [ ]:
X.shape

## 13.1Recombinaison des tables de données Train  et Test agregrées pour un preprocessing combiné

In [ ]:
# Combine application train and test tables for preprocessing.
Data = pd.concat([X, appli_test_agg], axis=0)

In [ ]:
Data.head()

In [ ]:
Data.shape

### 13.2 Quantification des valeurs manquantes

In [ ]:
# Missing Value
def missing_values(data, plot=False):
    mst = pd.DataFrame(
        {"Num_Missing": data.isnull().sum(), "Missing_Ratio": data.isnull().sum() / data.shape[0]}).sort_values(
        "Num_Missing", ascending=False)
    mst["DataTypes"] = data[mst.index].dtypes.values
    mst = mst[mst.Num_Missing > 0].reset_index().rename({"index": "Feature"}, axis=1)

    print("Number of Variables include Missing Values:", mst.shape[0], "\n")

    if mst[mst.Missing_Ratio >= 1.0].shape[0] > 0:
        print("Full Missing Variables:", mst[mst.Missing_Ratio >= 1.0].Feature.tolist())
        data.drop(mst[mst.Missing_Ratio >= 1.0].Feature.tolist(), axis=1, inplace=True)

        print("Full missing variables are deleted!", "\n")

    if plot:
        plt.figure(figsize=(25, 8))
        p = sns.barplot(mst.Missing_Ratio)
        for rotate in p.get_xticklabels():
            rotate.set_rotation(90)
        plt.show()

In [ ]:
#Quantification des valeurs manquantes
missing_values(Data, plot = True)

In [ ]:
gc.collect()

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
def null_factor(df, tx_threshold=30):
  null_rate = ((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False).reset_index()
  null_rate.columns = ['Variable','Taux_de_Null']
  high_null_rate = null_rate[null_rate.Taux_de_Null >= tx_threshold]
  return high_null_rate

In [ ]:
full_null_rate = null_factor(Data,30)
full_null_rate

# 14 Selection des features

## 14.1 Selection des colonnes avec des valeurs  nan inferieures ou ègal à 30%

In [ ]:
filling_features = null_factor(Data, 0)
filling_features["Taux_de_Null"] = 100-filling_features["Taux_de_Null"]
filling_features = filling_features.sort_values("Taux_de_Null", ascending=False) 

#Seuil de suppression
sup_threshold =70

fig = plt.figure(figsize=(20, 35))

font_title = {'family': 'serif',
              'color':  '#114b98',
              'weight': 'bold',
              'size': 18,
             }

sns.barplot(x="Taux_de_Null", y="Variable", data=filling_features, palette="flare")
#Seuil pour suppression des varaibles
plt.axvline(x=sup_threshold, linewidth=2, color = 'r')
plt.text(sup_threshold+2,70, 'Seuil de suppression des variables', fontsize = 16, color = 'r')

plt.title("Taux de remplissage des variables dans le jeu de données (%)", fontdict=font_title)
plt.xlabel("Taux de remplissage (%)")
plt.show()

In [ ]:
#Liste des variables à conserver
features_to_conserve = list(filling_features.loc[filling_features['Taux_de_Null']>=sup_threshold, 'Variable'].values)
#Liste des variables supprimées
deleted_features = list(filling_features.loc[filling_features['Taux_de_Null']<sup_threshold, 'Variable'].values)

#Nouveau Dataset avec les variables conservées
Data = Data[features_to_conserve]
Data.sample(5)

In [ ]:
Data.shape


## 14.2 Remplacement des nan par la valeur médiane dans chaque colonne

In [ ]:
df = Data.fillna(Data.median())

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
gc.collect()

In [ ]:
# Numeric Features
df.drop(["SK_ID_CURR" ], axis = 1).describe([.01, .1, .25, .5, .75, .8, .9, .95, .99])[1:]

In [ ]:
#Analyse univariées--->Application Train

In [ ]:
# Quick Visualization for numerical variables
#num_plot(df, num_cols=num_cols, remove=['SK_ID_CURR'], figsize = (15,3))

In [ ]:
# Analyses Multivariées---> Application Train

In [ ]:
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(10, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,cmap=cmap, center=0,
            square=True, linewidths=.5)

plt.title(f"Heatmap des corrélations linéaires\n", fontsize = 18)
plt.show()
fig.savefig("Heatmap des corrélations linéaires_P4_2.jpeg")

In [ ]:
threshold = 0.7
corr_pairs = corr.unstack().sort_values(kind="quicksort")
strong_corr = (pd.DataFrame(corr_pairs[(abs(corr_pairs) > threshold)])
               .reset_index().rename(columns={0:'corr_coeff'}))
strong_corr = strong_corr[(strong_corr.index%2 == 0) & (strong_corr['level_0'] != strong_corr['level_1'])]
strong_corr.sort_values('corr_coeff', ascending=False)

## 14.3 Suppression des variables correlées de type level_0 sans la variable ['CNT_CHILDREN']

In [ ]:
Drop_Col=strong_corr.level_0.values

In [ ]:
no_children = [x for x in Drop_Col if x != "CNT_CHILDREN"]

In [ ]:
df.drop(no_children,axis=1,inplace =True)

## 14.4 Suppression de la variable  de type level_1[ 'CNT_FAM_MEMBERS'] correlée  à la variable de type level_0 ['CNT_CHILDREN']

In [ ]:
df.drop('CNT_FAM_MEMBERS',axis=1,inplace=True)

## 14.5 Bilan et Evaluation de la taille de la dataframe df

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
gc.collect()

# 15 Séparation des données en jeux de données  X_train et X_test aggrégées

In [ ]:
# Split dataset back into its training and testing segments
X_train_agg = df[df['SK_ID_CURR'].isin(train_Sign)]
X_test_agg = df[df['SK_ID_CURR'].isin(test_Sign)]
X_test_agg.reset_index(drop=True, inplace=True)
X_train_agg = pd.merge(X_train_agg,y_train_agg, left_index=True, right_index=True)

In [ ]:
X_train_agg.head(3)

In [ ]:
X_test_agg.head(3)

# 16 Preprocessing avant sauvegarde des données aggrégées

## 16.1Suppression la variable Credit_flag contenant des données inexploitables

In [ ]:
X_train_agg=X_train_agg.drop('Credit_flag',axis=1)
X_test_agg=X_test_agg.drop('Credit_flag',axis=1)

## 16.2 Insertion de la variable Age 

In [ ]:
X_train_agg['Age']=(X_train_agg['DAYS_BIRTH']/365).round(1)
X_test_agg['Age']=(X_test_agg['DAYS_BIRTH']/365).round(1)

## 16.3 Suppression  de la variable DAYS_BIRTH

In [ ]:
X_train_agg=X_train_agg.drop('DAYS_BIRTH',axis=1)
X_test_agg=X_test_agg.drop('DAYS_BIRTH',axis=1)

## 16.4 Remplacement  de la colonne index par la colonne de type objet (SK_ID_CURR)

In [ ]:
df_train=X_train_agg.set_index('SK_ID_CURR')
df_test=X_test_agg.set_index('SK_ID_CURR')

# 17 Conversion de toutes les colonnes numerique  en colonne positive dans les datasets aggrégées avant sauvegarde.

In [ ]:
#df[df.columns[df.dtypes != np.object]] = df[df.columns[df.dtypes != np.object]].abs()
#print(df)
df_train[df_train.columns[df_train.dtypes !=np.object]]=df_train[df_train.columns[df_train.dtypes !=np.object]].abs()
df_train.head(3)

In [ ]:
df_test[df_test.columns[df_test.dtypes !=np.object]]=df_test[df_test.columns[df_test.dtypes !=np.object]].abs()
df_test.head(3)

# 18 Sauvegarde de train et test datasets aggrégées

In [ ]:
df_train.shape,df_test.shape

In [ ]:
df_train.to_csv('X_train_agg.csv',index=True)
df_test.to_csv('X_test_agg.csv',index=True)

# FIN D'ANALYSE EXPLORATION DES DONNEES EXTRAITES DES BASES DE DONNEES RELATIONNELLES